# Cox-Ingersoll-Ross Process

## 1. Introduction

Compared to the Ornstein-Uhlenbeck process, the Cox-Ingersoll-Ross (CIR) process allows for a stochastic diffusion component. This might be an attempt to improve the fit. To allow for sufficient comparison, the remaining aspects of the modelling process remain unchanged. As such, we once again use a MLE for the CIR process, which is defined as 

$$ dS_t = \alpha(\kappa - S_t)dt + \sigma \sqrt{dS_t} dW_t.$$

## 2. Maximum Likelihood Estimation 

In [1]:
import numpy as np

In [2]:
from qstrat.models.cir import CIR
from qstrat.fit.transition_density import ExactDensity
from qstrat.fit.mle_estimator import MLE
from qstrat.sim.simulator import SimulationSDE
from qstrat.plots.plot import plot_series_plotly, plot_density
from qstrat.utils.data_utils import (
    calculate_moments,
    conditional_expectation,
    expected_time_to_hit_target,
)
from qstrat.utils.data_utils import read_excel_to_series, strip_data

In [3]:
FILE_PATH = "../data/spread.xlsx"
COLUMN = "Spread"

In [4]:
df = read_excel_to_series(file_path=FILE_PATH)
spread = strip_data(df=df, column=COLUMN)

In [8]:
param_bounds = [(0, 50), (0, 50), (0.01, 50)]
guess = np.array([1, 0.1, 0.4])

In [9]:
freq = 252 * 12 * 8
dt = 1.0 / freq
model = CIR()

In [10]:
exact_est = MLE(
    sample=spread, param_bounds=param_bounds, dt=dt, density=ExactDensity(model=model)
).estimate_params(guess)

Initial Params: [1.  0.1 0.4]
Initial Likelihood: -126118.8457647932
`xtol` termination condition is satisfied.
Number of iterations: 53, function evaluations: 204, CG iterations: 135, optimality: 2.30e-01, constraint violation: 0.00e+00, execution time:  1.5 s.
Final Params: [0.05443819 0.74718357 1.7302386 ]
Final Likelihood: 44633.004627953196


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_differentiable_functions.py:231: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


In [8]:
params = exact_est.params
alpha, kappa, sigma = params[0], params[1], params[2]
alpha, kappa, sigma

(0.054438194235757915, 0.7471835728892986, 1.7302386007539634)

Based on the result of the MLE, we assume the spread can be fitted as an CIR with parameters 
$$ dS_t = \alpha(\kappa - S_t)dt + \sigma \sqrt{S_t} dW_t $$
$$ dS_t = 0.05(0.75 - S_t)dt + 1.73 \sqrt{S_t} dW_t $$ 

We can simulate this process using the Euler-Maruyama scheme, the most elementary discretization scheme as follows 
$$ S_{t_{n + 1}} = S_{t_{n}} + \alpha (\kappa - S_{t_{n}}) \Delta t  + \sigma \sqrt{\Delta t} \cdot \sqrt{S_t} \cdot \Delta W_n  $$ 

## 3. Plots

In [9]:
seed = 123
S0 = spread[0]
T = (len(spread) * 5) / ((252 * 8 * 60))
T = int(np.ceil((T)))

In [10]:
simulator = SimulationSDE(S0=S0, M=T * freq, dt=dt, model=model).set_seed(seed=seed)
sample = simulator.sim_path()
sample = sample.flatten()
sample

array([20.38      , 20.33749672, 20.3193283 , ..., 32.86133199,
       32.93936668, 32.90736501])

In [11]:
TITLE = "CIR Fit"
X_AXIS = "Time"
Y_AXIS = "Spread"
LABELS = ["Real Spread", "CIR"]

In [12]:
plot_series_plotly(
    spread,
    sample,
    title=TITLE,
    labels=LABELS,
    xaxis_title=X_AXIS,
    yaxis_title=Y_AXIS,
    truncate=True,
)

In [13]:
moments_spread = calculate_moments(data=spread)

{'Kurtosis': 0.39893613722928967,
 'Mean': 21.707308807655405,
 'Skewness': -0.5474354025181073,
 'Variance': 1.4148241583285703}


In [14]:
moments_sim = calculate_moments(data=sample)

{'Kurtosis': -0.5196684633474709,
 'Mean': 26.300899011156236,
 'Skewness': 0.526570368831798,
 'Variance': 25.93375649684989}


We note that once again, the fit is poor. Suprisingly, considering the fact that CIR has a non deterministic diffusion, one would expect a better fit than OU. Nevertheless the fit is clearly poorer than the previous fit. 

## 4. Summary Statistics

In the final section, we compute both the conditional expectation of the spread and the expected time to hit the long term average. Additionally, plots are also provided 

First, the conditional expectation of the spread to hit the long term average. We define the target as the mean of the spread. 

In [15]:
target_level = np.mean(spread)
target_level

21.707308807655405

Next, we set $T = 5 000$, noting that the data is a 5 minute spread. 

In [16]:
cond_exp = conditional_expectation(
    data=sample, starting_point=0, T=5000, target_level=target_level
)

[20.38       20.33749672 20.3193283  ... 22.94175708 22.97104544
 22.97532139]


Below, the plot for the conditional expectation of the spread for just a single path.

In [17]:
plot_density(
    values=cond_exp, title="Conditional Expectation of The Spread", xlabel="Time"
)

Next, for the expected time to hit the target level, i.e. the mean average, we run multiple paths and derive the expectime time in each path. 

In [18]:
NUM_PATHS = 1000

In [19]:
simulator = SimulationSDE(S0=S0, M=T * freq, dt=dt, model=model).set_seed(seed=seed)
sample = simulator.sim_path(num_paths=1000)

In [20]:
exp_time = expected_time_to_hit_target(
    data=sample, target_level=target_level, starting_point=0
)

In [21]:
plot_density(
    values=exp_time, title="Conditional Expected Termintation Time", xlabel="Time"
)

## 5. Conclusion 

We conclude our investigation by noting that using Maximum Likelihood Estimation to estimate the parameters of both the Ornstein-Uhlenbeck and CIR processes has demonstrated to be poor. In many cases, the estimations fail to capture the changes in momentum, although the fit does improve throughout time. Likewise, based on the summary statistics, the poorness of the fit are demonstrated when considering higher momenta such as the kurtosis and the skew. Both fits prove to be underestimating the heavy-tailed nature of the spread and overestimate the normality of it.

Finally, the optimization process is also highly sensitive to the initial guess and the parameter space. To remedy this deficiency, a possible much more intensive solution would be to consider a hyperparameter search (à la machine learning). Making a cross comparison between OU and CIR is deemed unnecessary since both models, in the context of MLE, return poor fits. 